# Generative Adversarial Networks

<img src="https://raw.githubusercontent.com/cherrerab/deeplearningfallas/master/workshop_08/bin/gan_sample_2.png" height="300">

## Raytracing Dataset

Para generar los datos o bien, las imágenes reales que utilizaremos para entrenar nuestro modelo, utilizaremos el algoritmo de `raytracing` desarrollado por James Bowman. Este algoritmo disponible en el github `https://github.com/mdoege/raytrace` nos permite renderizar esferas parametrizables en un espacio tridimensional, como se muestra en la animación.

![animation](https://github.com/mdoege/raytrace/raw/master/im.gif "animation")

De este modo, utilizaremos este programa para generar una serie de imágenes (`samples`) de `128x128px` que contengan una esfera en distintas posiciones en el espacio. Para ahorrar tiempo y concentrarnos en el desarrollo del modelo, este proceso ya ha sido realizado y el dataset `GAN_dataset_128px.npz` correspondiente ha sido cargado a un Google Drive.

In [ ]:
!pip install -U -q PyDrive

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# inicializar GoogleDrive con credenciales de autorización
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# crear carpeta para descargar los archivos .npz
!mkdir /content/datasets

# Google Drive IDs para descargar los archivos .npz
files_id = [('GAN_dataset_128px.npz', '1kSOTgEj9oSOXEb_2LkTQfkPdh3fFi5Eq')]

# comenzar descarga
print('descargando datasets: ', end='')

for filename, id in files_id:
  save_path = os.path.join('/content/datasets', filename)

  # descargar y guardar en /content/datasets
  downloaded = drive.CreateFile({'id': id}) 
  downloaded.GetContentFile(save_path)

# indicar descarga terminada
print('done')

Como ya es costumbre, carguemos este archivo mediante `np.load()` y exploremos las estructuras y datos que contiene.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ---
# cargar archivo GAN_dataset_128px.npz
dataset = np.load('/content/datasets/GAN_dataset_128px.npz', allow_pickle=True)

# print keys del dataset
print('dataset.keys: ',  list( dataset.keys() ) )

# ---
# extraer conjuntos de imágenes y normalizar en [0., 1.]
X = dataset['X']
X = X/255.0

# visualizar muestra del dataset
sample_idx = np.random.choice( np.arange(X.shape[0]), 5 )
img_sample = [ X[i, :, :, :].reshape( (128, 128, 3) ) for i in sample_idx ]
img_sample = np.hstack(img_sample)

plt.figure( figsize=(12, 12) )
plt.imshow(img_sample)

Así, como se mencionó anteriormente, el dataset cuenta con imágenes de `128x128px` con esferas de color variable que se posicionan aleatoriamente sobre el plano de ajedrez. De este modo, contamos con 2000 imágenes para el entrenamiento.

---
## Model Setup

En términos simples, un modelo GAN se compone de dos modelos independientes, el `Discriminador` y, por supuesto, el `Generador`.

<img src="https://raw.githubusercontent.com/cherrerab/deeplearningfallas/master/workshop_08/bin/GAN_diagram.png" width="600">

Por un lado, el `Discriminador`, como el nombre lo indica, está diseñado para discriminar las imágenes creadas artificalmente por el `Generador` de las imágenes reales contenidas en el dataset. De este modo, este modelo consiste simplemente en un modelo Convolucional de Clasificación (CNN), tal como estudiamos en el `workshop_03`.

Así, el `Discriminador` procesa secuencialmente la información contenida en la imagen de entrada mediante una serie de capas `Conv2D`, y posteriormente capas `Dense`, para finalmente retornar la etiqueta de clasificación de la imagen (`real` o `fake`) mediante una capa softmax.

<img src="https://raw.githubusercontent.com/cherrerab/deeplearningfallas/master/workshop_08/bin/DS_diagram.png" height="400">



In [ ]:
from keras.models import Model

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

# ---
# primero debemos configurar nuestra capa Input donde debemos especificar
# las dimensiones de los datos que se ingresarán al modelo
# en este caso el discriminador recibe las imágenes de (300, 300, 3)
input_dim = ( 128, 128, 3 )
discriminator_input = Input( shape=input_dim )

# ---
# ahora, como cualquier clasificador de imágenes, debemos ir agregando
# nuestras capas Conv2D y Pooling.

# las keras.layers.Conv2D reciben la cantidad de filtros dentro de la capa,
# el tamaño de estos filtros y la función de activación con que operarán.
# https://keras.io/api/layers/convolution_layers/convolution2d/

# las keras.layers.MaxPooling2D reciben el tamaño de la ventana sobre
# la cual llevarán a cabo el down-sampling
# https://keras.io/api/layers/pooling_layers/max_pooling2d/
discriminator = Conv2D(32, (5, 5), activation='relu', padding='same')(discriminator_input)
discriminator = Conv2D(32, (5, 5), activation='relu', padding='same')(discriminator)
discriminator = MaxPooling2D( pool_size=(2, 2) )(discriminator)

discriminator = Conv2D(64, (5, 5), activation='relu', padding='same')(discriminator)
discriminator = Conv2D(64, (5, 5), activation='relu', padding='same')(discriminator)
discriminator = MaxPooling2D( pool_size=(2, 2) )(discriminator)

discriminator = Conv2D(48, (3, 3), activation='relu', padding='same')(discriminator)
discriminator = Conv2D(48, (3, 3), activation='relu', padding='same')(discriminator)
discriminator = MaxPooling2D( pool_size=(2, 2) )(discriminator)

discriminator = Conv2D(32, (3, 3), activation='relu', padding='same')(discriminator)
discriminator = Conv2D(32, (3, 3), activation='relu', padding='same')(discriminator)
discriminator = MaxPooling2D( pool_size=(2, 2) )(discriminator)

# ---
# ahora debemos ir agregando nuestras capas Dense para procesar la
# información hasta la capa de salida.
# https://keras.io/api/layers/core_layers/dense/
discriminator = Flatten()(discriminator)
discriminator = Dropout( rate=0.2 )(discriminator)

discriminator = Dense(units=256, activation='relu')(discriminator)
discriminator = Dense(units=128, activation='relu')(discriminator)
discriminator = Dropout( rate=0.2 )(discriminator)

discriminator = Dense(units=128, activation='relu')(discriminator)
discriminator = Dense(units=64, activation='relu')(discriminator)

# ---
# por último, como siempre, debemos configurar nuestra capa de salida
# dado que el modelo discriminador consiste en simplemente un modelo de
# clasificación emplearemos la función softmax, donde cada nodo indicará la
# probabilidad de que los datos correspondan a una de las etiquetas.
labels_num = 2
discriminator_output = Dense(units=labels_num, activation='softmax')(discriminator)

# ---
# ahora configuraremos el modelo discriminador
DS = Model(discriminator_input, discriminator_output)

# print model.summary()
DS.summary()

Por el otro lado, el `Generador` está diseñado para construir imágenes a partir de un vector de ruido uniforme `NOISE_VECTOR`. Por supuesto, el tamaño de este vector o `NOISE_DIM` será otro de los hiperparámetros que constituirá el modelo.

De este modo, de manera inversa a un modelo Convolucional convencional, el `Generador` procesa inicialmente este vector de ruido mediante una serie de capas `Dense` para luego dar paso a una reconfiguración del vector latente resultante a una forma compatible con las estructuras convolucionales `(n_sample, height, width, filters)` mediante una capa `Reshape`. Así, de forma simétrica a la estructura del `Discriminador`, una serie de capas `Conv2DTranspose` y `UpSampling2D` se encargan de ir construyendo secuencialmente una imagen RGB final de `128x128px` compatible con las características de las imágenes reales. Para compilar esta última imagen, la salida de este modelo se compone de una capa `Conv2D` de tres filtros (tres canales RGB).

<img src="https://raw.githubusercontent.com/cherrerab/deeplearningfallas/master/workshop_08/bin/GN_diagram.png" width="600">

In [ ]:
from keras.models import Model

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Dropout
from keras.layers import Conv2DTranspose
from keras.layers import UpSampling2D

# ---
# en el caso del modelo generador, este recibe como input el vector latente de
# ruido a partir del cual construirá la imagen.
# de este modo la dimensión de entrada de la capa Input depende del tamaño
# de este vector noise.
input_dim = ( 128, )
generator_input = Input( shape=input_dim )

# ---
# ahora para transformar el vector noise en una imagen primero es necesario
# agregar capas Dense con la finalidad de obtener un vector que puede ser
# reconfigurado como imagen.
# https://keras.io/api/layers/core_layers/dense/
generator = Dense(units=128, activation='relu')(generator_input)
generator = Dense(units=256, activation='relu')(generator)

generator = Dense(units=8*8*32, activation='relu')(generator)
generator = Reshape(target_shape=(8, 8, 32))(generator)

# para generar o bien, construir la imagen de salida, utilizaremos capas
# Conv2DTranspose y UpSampling2D hasta alcanzar las mismas características
# que las imágenes del dataset.
# https://keras.io/api/layers/convolution_layers/convolution2d_transpose/
# https://keras.io/api/layers/reshaping_layers/up_sampling2d/
generator = UpSampling2D( size=(2, 2) )(generator)
generator = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(generator)
generator = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(generator)

generator = UpSampling2D( size=(2, 2) )(generator)
generator = Conv2DTranspose(48, (3, 3), activation='relu', padding='same')(generator)
generator = Conv2DTranspose(48, (3, 3), activation='relu', padding='same')(generator)

generator = UpSampling2D( size=(2, 2) )(generator)
generator = Conv2DTranspose(64, (5, 5), activation='relu', padding='same')(generator)
generator = Conv2DTranspose(64, (5, 5), activation='relu', padding='same')(generator)

generator = UpSampling2D( size=(2, 2) )(generator)
generator = Conv2DTranspose(64, (5, 5), activation='relu', padding='same')(generator)
generator = Conv2DTranspose(64, (5, 5), activation='relu', padding='same')(generator)

# ---
# finalmente, utilizaremos una capa Conv2D de tres filtros para generar
# la imagen de salida.
generator_output = Conv2D(3, (5, 5), activation='linear', padding='same')(generator)

# ---
# ahora configuraremos el modelo discriminador
GN = Model(generator_input, generator_output)

# print model.summary()
GN.summary()

Ahora al compilar estos modelos, hay que tener presente que el `Discriminador` debe ser entrenado aisladamente, sin conexión con el `Generador`, tal como se muestra en su diagrama.

Por otro lado, dado que el objetivo del `Generador` es confundir al `Discriminador`, su función de pérdida (`loss function`) está directamente ligada al output del `Discriminador`. Así, para su entrenamiento, el `Generador` debe ser compilado en conjunto con el `Discriminador`, pero con los pesos de éste último congelados. De este modo, la entrada de este modelo compuesto (i.e la GAN) es el vector de ruido (`NOISE_VECTOR`) del `Generador`, mientras que la salida corresponde a la probabilidad de clasificación computada por el `Discriminador`.

In [ ]:
from keras.optimizers import Adam
from keras.models import Sequential

# ---
# parámetros de la GAN
NOISE_DIM = 128
IMG_DIM = (128, 128, 3)

# ---
# compilar modelo discriminador
opt = Adam( learning_rate=1e-4 )
DS.compile(loss='binary_crossentropy', optimizer=opt, metrics=['mae'])
    
# ---
# construir modelo GAN
GAN = Sequential()
GAN.add(GN)
GAN.add(DS)

# congelar el entrenamiento del discriminador
DS.trainable = False

# compilar modelo GAN
GAN.compile(loss='binary_crossentropy', optimizer=opt, metrics=['mae'])

## Model Training

Dado que internamente los modelos que componen a la GAN presentan objetivos distintos, el entrenamiento de esta arquitectura requiere de un entrenamiento secuencial iterativo, que permita el entrenamiento aislado tanto del `Discriminador` como del `Generador`.

<img src="https://raw.githubusercontent.com/cherrerab/deeplearningfallas/master/workshop_08/bin/GAN_diagram.png" width="600">

En primer lugar, al inicio de una iteración de entrenamiento, el `Generador` es utilizado para generar un cantidad `batch_size` de imágenes falsas. Posteriormente, este `batch` de imágenes falsas es combinado con un `batch` de imágenes reales extraídas del dataset original, para luego ser alimentado al entrenamiento del `Discriminador`. Como cualquier otro clasificador, el `Discriminador` es entrenado siguiendo una `binary_crossentropy` para aprender a discernir entre los dos sets de datos.

Una vez que el `Discriminador` ha sido entrenado sobre este `batch` de imágenes, este es congelado para el entrenamiento del `Generador`. A parir de un `batch` de vectores de ruido uniforme el `Generador` es entrenado para generar imágenes que confundan al `Discriminador` de la GAN. Es decir, la función de pérdida del `Generador` apunta a que el `Discriminador` clasifique como `reales` las imágenes generadas artificialmente.

Terminado el entrenamiento del `Generador`, los pesos del `Discriminador` son restaurados como entrenables y el proceso se repite hasta finalizar con las épocas de entrenamiento especificadas.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from IPython.display import clear_output
from progressbar import ProgressBar
from keras.utils import to_categorical

# ---
# configurar parámetros de entrenamiento

# número de updates para cada iteración de entrenamiento
GLOBAL_UPDATES = 100
DS_UPDATES = 20
GN_UPDATES = 20

# batch_size para cada update de entrenamiento
BATCH_SIZE = 64
NUM_EPOCHS = 200

# parámetro de monitoreo
MONITOR_IT = 1

# ---
# función de extracción de imágenes reales para generar los batches
def get_real_images(X, batch_size):
  """
  -> np.array()

  extrae de manera aleatoria 'batch_size' imágenes del dataset X.

  :param np.array X:
    dataset que contiene las imágenes a extraer.
  :param int batch_size:
    cantidad de imágenes a extraer.
  
  :returns: np.array de la forma (batch_size, height, width, channels) 
  """

  # obtener index a extraer aleatoriamente
  samples_idx = np.random.choice( np.arange(X.shape[0]), batch_size)

  # obtener las imágenes para el batch
  X_real = X[samples_idx, :, :, :]
  return X_real

# ---
# inicializar listas de función de pérdida
avg_loss_DS = []
avg_loss_GN = []
total_it = 0

# training loop
for epoch in range(NUM_EPOCHS):

  # inicializar listas internas
  loss_DS = []
  loss_GN = []

  for it in range(GLOBAL_UPDATES):
    # ---
    # discriminator training loop
    DS.trainable = True
    bar = ProgressBar()
    for i in bar( range(DS_UPDATES) ):
      # obtener batch de imágenes reales (1)
      X_real = get_real_images(X, BATCH_SIZE)
      Y_real = np.ones( (BATCH_SIZE, 1) )

      # generar batch de imágenes falsas (0)
      noise_vector = np.random.randn(BATCH_SIZE, NOISE_DIM)
      X_fake = GN.predict(noise_vector)
      Y_fake = np.zeros( (BATCH_SIZE, 1) )

      # compilar batch para entrenamiento
      X_DS = np.vstack([X_real, X_fake])
      Y_DS = np.vstack([Y_real, Y_fake])
      Y_DS = to_categorical(Y_DS, num_classes=2)

      batch_shuffle = np.random.permutation( np.arange(2*BATCH_SIZE) )
      X_DS = X_DS[batch_shuffle, :, :, :]
      Y_DS = Y_DS[batch_shuffle, :]

      # entrenar sobre el batch de imágenes, mediante train_on_batch
      DS_loss = DS.train_on_batch( X_DS, Y_DS )[1]

    # ---
    # visualizar imágenes de muestra para monitorear el entrenamiento
    if (total_it % MONITOR_IT == 0):
      # obtener 5 imágenes reales
      imgs_real = get_real_images(X, 5)

      # generar 9 imágenes falsas
      noise_vec = np.random.randn(9, NOISE_DIM)
      imgs_fake = GN.predict(noise_vec)

      # para cada set de imágenes
      for img_set in [imgs_fake, imgs_real]:
        plt.figure( figsize=(15, 3) )

        for i in range(5):
          # obtener imagen
          img = img_set[i, :, :, :].reshape( (128, 128, 3) )
          img = np.clip(img, 0.0, 1.0)

          # obtener clasificación del discriminador
          x = img.reshape( (1, 128, 128, 3) )
          DS_pred = DS.predict(x)[0, 1]

          # visualizar
          plt.subplot(1, 5, i+1)
          plt.title( 'score: {:1.2f}'.format(DS_pred) )
          plt.imshow(img)
      clear_output(True)
      plt.show()

      # ---
      # generator training loop
      DS.trainable = False
      GN_loss = 0

      bar = ProgressBar()

      for i in bar( range(GN_UPDATES) ):
        # generar batch de imágenes falsas
        X_GN = np.random.randn(BATCH_SIZE, NOISE_DIM)
        Y_GN = np.ones( (BATCH_SIZE, 1) )
        Y_GN = to_categorical(Y_GN, num_classes=2)

        # entrenar sobre el batch de imágenes, mediante train_on_batch
        GN_loss += GAN.train_on_batch(X_GN, Y_GN)[1]

    # ---
    # registrar training_history
    loss_DS.append( DS_loss )        
    loss_GN.append( GN_loss/GN_UPDATES )
    total_it += 1

  # ---
  # visualizar función de pérdida
  avg_loss_DS.append( np.mean(loss_DS) )
  avg_loss_GN.append( np.mean(loss_GN) )

  plt.figure( figsize=(15, 5) )
  plt.plot( range(len(avg_loss_DS)), avg_loss_DS )
  plt.plot( range(len(avg_loss_GN)), avg_loss_GN )
  plt.legend(['discriminator loss', 'generator loss'])
  plt.show()